This notebook continuously estimates the prevailing discrete/realtime positivity values via 

* $positives_{c, \small{100K}, \tau} \;\; vs.\ \; tests_{c, \small{100K}, \tau}$

curves, wherein

$\qquad \qquad \qquad positives_{c, \small{100K}, \tau} \; = \; \small{100\,000} \times \displaystyle \left(\sum_{\substack{
   start \\
   date
  }}^{\tau}{P_{\delta}}  \right) \times \frac{1}{population}$


$\qquad \qquad \qquad \qquad  tests_{c, \small{100K}, \tau} \; = \; \small{100\,000} \times \displaystyle \left(\sum_{\substack{
   start \\
   date
  }}^{\tau}{T_{\delta}} \right) \times \frac{1}{population}$  

<br>

which are the positive cases per hundred thousand and test numbers per hundred thousand, respectively, based on their cumulative values and

<br>

<table style="width:35%; text-align: left; border: 0px solid black; float:left; margin-left: 50px">
    <tr>
        <th style="width:20%;">$\mathcal{Variable} \qquad$</th><th></th> 
    </tr>
    <tr>
        <td>$\tau$</td><td>end date</td>
    </tr>
    <tr>
        <td>$\delta$</td><td>a series date</td>
    </tr>
    <tr>
      <td>$P_{\delta}$</td><td>The number of positive cases on date $\delta$.</td>
    </tr>
    <tr>
      <td>$T_{\delta}$</td><td>The number of tests on date $\delta$.</td>
    </tr>
    
</table>


<br>

<br>

The subscript $c$ denotes cumulative.   The graph image below illustrates the curves in question.  In brief 

* The quotient of the coordinates of a point along a curve is the cumulative positivity value. 

* The gradient of a modelled secant line to a sequential set of curve rear points estimates the prevailing positivity value.

* The aforementioned secant line model predicts the expected values of $positives_{c, \small{100K}}$ as $tests_{c, \small{100K}}$ increases.


<div style="margin-left: 100px;">
    <img src="https://github.com/briefings/sars/raw/develop/fundamentals/atlantic/docs/ptr.png" longdesc="surveillance" width="40%" style="float:center">
</div>

<br>

Upcoming: Link this image to the interactive graph.



<br>
<br>

## Preliminaries: In Progress

### Libraries

In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np
import dask

import logging

import os
import pathlib
import sys

import statsmodels.regression.linear_model


In [2]:
from statsmodels.formula.api import ols

<br>

### Paths

In [3]:
child = os.getcwd()
parent = str(pathlib.Path(child).parent)

In [4]:
root = os.path.join(child, 'warehouse')
warehouse = os.path.join(root, 'prospects')

<br>

Appending paths

In [5]:
sys.path.append(parent)

<br>

### Logging

In [6]:
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

<br>

### Custom

Import

In [7]:
import cartographs.boundaries.us.boundaries
import cartographs.boundaries.us.settings

In [8]:
import algorithms.curves.estimates
import algorithms.curves.predictions
import algorithms.curves.secants

In [9]:
import atlantic.base.directories
import atlantic.explore.prospects
import atlantic.explore.hospitalisations

<br>

Set-up directories

In [10]:
directories = atlantic.base.directories.Directories()
directories.cleanup(listof=[warehouse])
directories.create(listof=[warehouse])

<br>
<br>

### Classes

**Boundaries**

In [11]:
settings = cartographs.boundaries.us.settings.Settings()
boundaries = cartographs.boundaries.us.boundaries.Boundaries(settings.crs)

In [12]:
states = boundaries.states(year=settings.latest)
states.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 56 entries, 0 to 55
Data columns (total 10 columns):
 #   Column    Non-Null Count  Dtype   
---  ------    --------------  -----   
 0   STATEFP   56 non-null     object  
 1   STATENS   56 non-null     object  
 2   AFFGEOID  56 non-null     object  
 3   GEOID     56 non-null     object  
 4   STUSPS    56 non-null     object  
 5   NAME      56 non-null     object  
 6   LSAD      56 non-null     object  
 7   ALAND     56 non-null     int64   
 8   AWATER    56 non-null     int64   
 9   geometry  56 non-null     geometry
dtypes: geometry(1), int64(2), object(7)
memory usage: 4.5+ KB


<br>
<br>

## Data

### Attributes

In [13]:
fields = ['datetimeobject', 'STUSPS', 'positiveIncrease', 'testIncrease', 'deathIncrease', 'hospitalizedIncrease', 
          'positiveCumulative', 'testCumulative',  'deathCumulative', 'hospitalizedCumulative', 
          'positiveRate', 'testRate', 'deathRate', 'hospitalizedRate', 'ndays']

dtype = {'STUSPS': 'str', 'positiveIncrease': np.float64, 'testIncrease': np.float64, 'deathIncrease': np.float64, 'hospitalizedIncrease': np.float64,          
         'positiveCumulative': np.float64, 'testCumulative': np.float64, 'deathCumulative': np.float64, 'hospitalizedCumulative': np.float64, 
         'positiveRate': np.float64, 'testRate': np.float64, 'deathRate': np.float64, 'hospitalizedRate': np.float64, 'ndays': np.int64}        

parse_dates = ['datetimeobject']

<br>

### Source URL

In [14]:
datauri = os.path.join(parent, 'warehouse', 'baselines.csv')

<br>

### Data

In [15]:
baselines = pd.read_csv(filepath_or_buffer=datauri, header=0, usecols=fields, 
                        dtype=dtype, encoding='utf-8', parse_dates=parse_dates)

baselines.loc[:, 'positiveTestRate'] = np.where(baselines['testRate'] > 0, 100 * baselines['positiveRate'] / baselines['testRate'], 0)

logger.info(baselines.info())

INFO:__main__:None


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12948 entries, 0 to 12947
Data columns (total 16 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   datetimeobject          12948 non-null  datetime64[ns]
 1   STUSPS                  12948 non-null  object        
 2   deathIncrease           12948 non-null  float64       
 3   deathCumulative         12948 non-null  float64       
 4   positiveIncrease        12948 non-null  float64       
 5   positiveCumulative      12948 non-null  float64       
 6   hospitalizedIncrease    12948 non-null  float64       
 7   hospitalizedCumulative  12948 non-null  float64       
 8   testIncrease            12948 non-null  float64       
 9   testCumulative          12948 non-null  float64       
 10  deathRate               12948 non-null  float64       
 11  positiveRate            12948 non-null  float64       
 12  testRate                12948 non-null  float6

<br>
<br>

## Regression

### $\small{Set-up}$

#### Latest Date

In [16]:
latestdate = baselines.datetimeobject.max()
logger.info(type(latestdate))
logger.info(latestdate)

INFO:__main__:<class 'pandas._libs.tslibs.timestamps.Timestamp'>
INFO:__main__:2020-09-26 00:00:00


<br>
<br>

#### Periods

Within the context of the aforementioned $positives_{c, \small{100K}, \tau} \;\; vs.\ \; tests_{c, \small{100K}, \tau}$ curves, each value of this array is the number of data points, starting from the end of a curve, that are used to develop **a regression model of the tangent/secant line to the points**.

Key points:

* The model predicts the expected $positives_{c, \small{100K}}$ as $tests_{c, \small{100K}, \tau}$ increases.

* The gradient of each model is an estimate of the prevailing positivity rate; an estimate of the current daily positivity value. [cf. continous positivity value] 

In [17]:
periods = np.arange(8, 29)
logger.info( periods)

INFO:__main__:[ 8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28]


<br>
<br>

#### Partitions

In [18]:
partitionby = 'STUSPS'
partitions = baselines[partitionby].unique()
logger.info(partitions)

INFO:__main__:['AL' 'AK' 'AZ' 'AR' 'CA' 'CO' 'CT' 'DE' 'DC' 'FL' 'GA' 'HI' 'ID' 'IL'
 'IN' 'IA' 'KS' 'KY' 'LA' 'ME' 'MD' 'MA' 'MI' 'MN' 'MS' 'MO' 'MT' 'NE'
 'NV' 'NH' 'NJ' 'NM' 'NY' 'NC' 'ND' 'OH' 'OK' 'OR' 'PA' 'RI' 'SC' 'SD'
 'TN' 'TX' 'UT' 'VT' 'VA' 'WA' 'WV' 'WI' 'WY' 'PR']


<br>
<br>

### $positives$ $\quad vs.\ \;$ $tests$

#### Analysis

In [19]:
formula = 'positiveRate ~ testRate'
regressor = 'testRate'

prospects = atlantic.explore.prospects.Prospects(data=baselines, partitionby=partitionby, periods=periods, 
                                                 formula=formula, regressor=regressor, step=100, num=5, latestdate=latestdate)

In [20]:
estimates, predictions = prospects.exc(partitions=baselines[partitionby].unique())

<br>

Estimates

In [21]:
logger.info(estimates.info())

INFO:__main__:None


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52 entries, 0 to 51
Data columns (total 8 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   datetimeobject           52 non-null     datetime64[ns]
 1   gradient                 52 non-null     object        
 2   lowerconfidenceinterval  52 non-null     object        
 3   upperconfidenceinterval  52 non-null     object        
 4   pvalue                   52 non-null     object        
 5   rsquared                 52 non-null     object        
 6   intercept                52 non-null     object        
 7   STUSPS                   52 non-null     object        
dtypes: datetime64[ns](1), object(7)
memory usage: 3.4+ KB


<br>

Predictions

In [22]:
logger.info(predictions.head())

INFO:__main__:   testRate      minimum       median      maximum STUSPS
0   22900.0  3095.952945  3105.825293  3123.446414     AL
1   23000.0  3108.408812  3119.627244  3138.721376     AL
2   23100.0  3120.864680  3133.429196  3153.996338     AL
3   23200.0  3133.320548  3147.231147  3169.271299     AL
4   23300.0  3145.776415  3161.033099  3184.546261     AL


<br>

#### State of affairs

In [23]:
ptr = estimates

ptr.loc[:, 'prevailingPTR'] = 100 * ptr['gradient']
ptr = ptr.merge(baselines[['datetimeobject', 'STUSPS', 'positiveTestRate']], how='left', on=['datetimeobject', 'STUSPS'])
ptr.loc[:, 'cf'] = ptr[['prevailingPTR', 'positiveTestRate']].max(axis=1)

ptr.loc[:, 'rank'] = ptr['cf'].rank(method='min', ascending=False)
ptr.sort_values(by='rank', ascending=True, inplace=True)
ptr.drop(columns=['cf'], inplace=True)

ptr.loc[:, 'prevailingTPC'] = np.where(ptr['prevailingPTR'] > 0, ptr['prevailingPTR'].rdiv(100), 0)
ptr.rename(columns={'lowerconfidenceinterval': 'gradientLCI', 'upperconfidenceinterval': 'gradientUCI', 'positiveTestRate': 'continuousPTR'})
ptr.to_csv(path_or_buf=os.path.join(warehouse, 'ptr.csv'), index=False, header=True, encoding='utf-8')

ptr.tail()

,datetimeobject,gradient,lowerconfidenceinterval,upperconfidenceinterval,pvalue,rsquared,intercept,STUSPS,prevailingPTR,positiveTestRate,rank,prevailingTPC
31,2020-09-26,0.022689,0.0213131,0.0300267,1.04534e-18,0.993072,384.971,NM,2.2689,3.188288,48.0,44.0741
29,2020-09-26,0.0168951,0.00417493,0.0219861,1.37836e-08,0.87337,280.863,NH,1.68951,3.060163,49.0,59.1888
1,2020-09-26,0.027061,0.0231871,0.0351657,1.94134e-16,0.984344,-511.73,AK,2.7061,1.877530,50.0,36.9536
19,2020-09-26,0.00517411,0.00314699,0.0068647,2.58415e-15,0.981615,234.487,ME,0.517411,1.294231,51.0,193.27
45,2020-09-26,0.00522171,0.00361079,0.00664267,4.1158e-12,0.96501,144.188,VT,0.522171,1.085003,52.0,191.508


<br>
<br>

### $hospitalisations$ $\; vs.\ \;$ $positives$

In [24]:
del formula, regressor, prospects, estimates, predictions

<br>

#### Data

In [25]:
hospitalisations = atlantic.explore.hospitalisations.Hospitalisations(blob=baselines, warehouse=os.path.join(parent, 'warehouse'))

In [26]:
hospital = hospitalisations.exc(limit=periods.min())

INFO:atlantic.explore.hospitalisations:
Hospitalized Positives Curves:
None



<class 'pandas.core.frame.DataFrame'>
Int64Index: 8964 entries, 0 to 8963
Data columns (total 6 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   datetimeobject            8964 non-null   datetime64[ns]
 1   STUSPS                    8964 non-null   object        
 2   hospitalizedRate          8964 non-null   float64       
 3   positiveRate              8964 non-null   float64       
 4   ndays                     8964 non-null   int64         
 5   hospitalizedPositiveRate  8964 non-null   float64       
dtypes: datetime64[ns](1), float64(3), int64(1), object(1)
memory usage: 490.2+ KB


<br>

#### Analysis

In [27]:
formula = 'hospitalizedRate ~ positiveRate'
regressor = 'positiveRate'

prospects = atlantic.explore.prospects.Prospects(data=baselines, partitionby=partitionby, periods=periods, 
                                                 formula=formula, regressor=regressor, step=10, num=5, latestdate=latestdate)

In [28]:
estimates, predictions = prospects.exc(partitions=hospital[partitionby].unique())

<br>

#### State of Affairs

In [29]:
hpr = estimates

<br>

Prevailing Hospitalised Positive Rate

In [30]:
hpr.loc[:, 'prevailingHPR'] = 100 * hpr['gradient']
hpr.loc[:, 'rank'] = hpr['prevailingHPR'].rank(method='min', ascending=False)
hpr.sort_values(by='rank', ascending=True, inplace=True)

<br>

Structuring

In [31]:
hpr = hpr.merge(hospital[['datetimeobject', 'STUSPS', 'hospitalizedPositiveRate']], how='left', on=['datetimeobject', 'STUSPS'])
hpr.loc[:, 'prevailingCPH'] = np.where(hpr['prevailingHPR'] > 0, hpr['prevailingHPR'].rdiv(100), 0)

hpr.rename(columns={'lowerconfidenceinterval': 'gradientLCI', 'upperconfidenceinterval': 'gradientUCI', 'hospitalizedPositiveRate': 'continuousHPR'}, 
           inplace=True)

hpr.to_csv(path_or_buf=os.path.join(warehouse, 'hpr.csv'), index=False, header=True, encoding='utf-8')

hpr.tail()

,datetimeobject,gradient,gradientLCI,gradientUCI,pvalue,rsquared,intercept,STUSPS,prevailingHPR,rank,continuousHPR,prevailingCPH
31,2020-09-26,0.0275891,0.0140452,0.0316807,1.63669e-14,0.97685,18.8257,WY,2.75891,32.0,4.720952,36.2463
32,2020-09-26,0.0272553,-0.00523064,0.053471,9.4752e-08,0.839216,100.831,MS,2.72553,33.0,5.799725,36.6901
33,2020-09-26,0.025932,0.0200611,0.0413606,2.12845e-14,0.976079,59.193,NE,2.5932,34.0,5.273157,38.5624
34,2020-09-26,0.0240162,0.0181873,0.0664727,1.68903e-09,0.908439,39.4621,NH,2.40162,35.0,9.038296,41.6385
35,2020-09-26,-4.44089e-16,-8.1008e-13,8.15409e-13,0.939291,-6352.5,462.615,NY,-4.44089e-14,36.0,19.789559,0


<br>
<br>

## Clean-up

In [32]:
!rm -rf *.log
!rm -rf *.pdf
!rm -rf states